In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

In [2]:
df = pd.read_csv('train.csv')
df1 = pd.read_csv('test.csv')
df = df.sample(frac=1).reset_index(drop=True)
#print len(df[df.loan_status==0]),len(df[df.loan_status==1])#532428.0

def generate(ind=None):
    df = pd.read_csv('train.csv')
    df1 = df[df.loan_status==0].sample(frac=1).reset_index(drop=True)
    df2 = df[df.loan_status==1]
    x = np.linspace(0,406601,4).astype(int)
    return pd.concat([df1.iloc[x[ind-1]:x[ind]],df2],axis=0).reset_index(drop=True)

In [3]:
### excluding low expected and object type features
def get_train_test():
    df_train = pd.read_csv('train.csv')
    df_test = pd.read_csv('test.csv')
    df = pd.concat([df_train,df_test],ignore_index=True)
    '''cols = [col for col in df.columns if df[col].dtype=='object']
    df.drop(cols,axis=1,inplace=True)
    cols = df.columns.tolist()
    print df.info()
    for col in cols:
        print col+"-"*5+str(df[col].dtype)+"\n"
        drop = input()
        if drop==1:
            df.drop(col,axis=1,inplace=True)'''
    return df[["acc_now_delinq",'annual_inc','collection_recovery_fee','delinq_2yrs','dti',
              'funded_amnt','inq_last_6mths','int_rate','open_acc','pub_rec','tot_coll_amt',
               'tot_cur_bal','total_acc','loan_status','revol_bal']]

def get_duo():
    return pd.read_csv('duo.csv')

df = get_duo()

In [4]:
#df['revol_bal'] = pd.concat([pd.read_csv('train.csv'),pd.read_csv('test.csv')],ignore_index=True)['revol_bal']
#df['loan_status'] = pd.concat([pd.read_csv('train.csv'),pd.read_csv('test.csv')],ignore_index=True)['loan_status']
#df.drop(['revol_util'],axis=1,inplace=True)

In [5]:
#### insert mean values with one or more missing values
labels = df[df['loan_status'].notnull()]['loan_status'].values
df.drop(['loan_status'],axis=1,inplace=True)
cols = df.columns

for col in cols:
    if (df[col].isnull().any()):
        print col
        print 
        df[col].fillna(df[col].mean(),inplace=True)

annual_inc

tot_cur_bal

total_acc

annual%laon_amnt



In [24]:
#### seperate dataframnes
leng = len(labels)
df_train = df.iloc[:leng]
df_test = df.iloc[leng:]
xtrain = df_train.values
xtest = df_test.values
clf = RandomForestClassifier(min_samples_leaf=40, n_estimators=60)
clf.fit(xtrain,labels)
pred = clf.predict_proba(xtest)
member_id = pd.read_csv('test.csv').member_id.values
df_sub = pd.DataFrame({"member_id":member_id,
                     'loan_status':pred[:,1]}
                    )


In [25]:
print len(member_id)
print len(pred[:,1])

def frmt(item):
    return float(format(item,'.4f'))

def features_imp(clf):
    imp = map(frmt,clf.feature_importances_)
    feat = list(df_train.columns)
    df = pd.DataFrame({'features': feat,
                       'importances': imp
                      })
    return df

features_imp(clf).sort(['importances'],ascending=False)


354951
354951


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:15: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,features,importances
53,mem_id_high,0.2742
57,mem_id_lowest,0.1086
56,mem_id_low3,0.0862
4,tot_cur_bal,0.0480
55,mem_id_low2,0.0423
28,initial_list_status_f,0.0382
54,mem_id_low1,0.0362
31,recovery_low,0.0350
30,recovery_high0,0.0340
58,mem_id_medium,0.0327


In [26]:
print df_sub['loan_status'].describe()

count    354951.000000
mean          0.240776
std           0.253559
min           0.000000
25%           0.052737
50%           0.149385
75%           0.339793
max           0.984730
Name: loan_status, dtype: float64


In [29]:
df_sub.drop(['loan_status'],axis=1,inplace=True)
df_sub['loan_status'] = pred[:,1]
df_sub.loc[ (df_sub.loan_status==0) | (df_sub.loan_status==1),'loan_status'] = np.array(0.00001)
df_sub.to_csv("submission_new1.csv",index=False)
print df_sub['loan_status'].describe()

# df_sub.drop(['loan_status'],axis=1,inplace=True)
# df_sub['loan_status'] = np.array([0.0]*354951)
# df_sub.to_csv("submission1.csv",index=False)

count    354951.000000
mean          0.240776
std           0.253559
min           0.000010
25%           0.052737
50%           0.149385
75%           0.339793
max           0.984730
Name: loan_status, dtype: float64


In [29]:
df_sub['loan_status'] = df_sub['loan_status'].map(frmt)
df_sub.to_csv("submission1_new.csv",index=False)

In [11]:
df_sub['loan_status'].describe()

count    354951.000000
mean          0.240804
std           0.242210
min           0.007917
25%           0.063721
50%           0.158893
75%           0.335733
max           0.971190
Name: loan_status, dtype: float64

In [12]:
### selection of algo
### spliting training set into cv,test and train



clf_rf = RandomForestClassifier()
clf_lr = LogisticRegression()
clf_nb = GaussianNB()
x,x_test,y,y_test = train_test_split(xtrain,labels,test_size=0.2,train_size=0.8)
x_train,x_cv,y_train,y_cv = train_test_split(x,y,test_size = 0.1,train_size=0.9)




# for clf in [clf_rf,clf_lr,clf_nb]:
#     print cross_val_score(clf,x_train,y_train,scoring="f1").mean()

In [10]:
for clf in [clf_rf,clf_lr,clf_nb]:
    print cross_val_score(clf,x_train,y_train,scoring="roc_auc").mean()

0.746752154837
0.624499086462
0.621492627056


In [13]:
print "up here 1"
### since random forest serves best for our problem
### using gridsearchcv for tunning classifier arguments




para = {
    'n_estimators' : [10,20,40,60,80],
    'min_samples_leaf' : [40,50,70,90],
}
clf_rf.get_params().keys()
clf_gs = GridSearchCV(estimator = clf_rf, param_grid = para, cv= 5, scoring = "roc_auc")
clf_gs.fit(x_train,y_train)
clf_gs.best_params_

up here 1


{'min_samples_leaf': 40, 'n_estimators': 60}

In [ ]:
df_sol1 = 